In [2]:
import os
import pickle
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.layers import LSTM, Input, Dense, Activation, Embedding, Concatenate, Reshape
from keras.layers import RepeatVector, Permute
from keras.layers import Multiply, Lambda
import keras.backend as K 
from keras.models import Model
from keras.optimizers import RMSprop
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint, EarlyStopping

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

2023-05-21 22:24:47.288575: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-21 22:24:50.846738: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-05-21 22:24:50.846928: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/loca

Num GPUs Available:  0


2023-05-21 22:24:54.689923: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-05-21 22:24:54.689969: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-05-21 22:24:54.690010: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (umn-1684688989): /proc/driver/nvidia/version does not exist


### Load data from the Oskar Kolberg's Dataset
https://webesac.pcss.pl/

In [3]:
data = []

In [4]:
files = ["data_kolberg_with_bars"]
for file in files:
    with open('{}.pkl'.format(file), 'rb') as handle:
        data.extend(pickle.load(handle))

In [5]:
number_of_pieces = len(data)
number_of_pieces

19092

### Remove too long or too short phrases

Indicate the range of the most common lengths of phrases

In [6]:
notes_data = []
durations_data = []
bars_data = [] 
phrases_data = []

for piece in data:
    notes_data.extend(piece[0])
    durations_data.extend(piece[1])
    bars_data.extend(piece[2])
    phrases_data.extend(piece[3])

In [7]:
print(data[0][0])
print(data[0][1])
print(data[0][2])
print(data[0][3])

[60, 62, 64, 65, 67, 69, 67, 65, 65, 65, 65, 67, 65, 64, 60, 62, 64, 65, 67, 69, 67, 65, 65, 65, 65, 67, 65, 64, 65, 65, 64, 62, 64, 64, 62, 62, 60, 59, 57, 55, 60, 62, 64, 65, 67, 64, 62, 60, 128]
[0.25, 0.25, 0.5, 0.5, 1.0, 0.25, 0.25, 0.25, 0.25, 0.5, 0.5, 0.25, 0.25, 1.0, 0.25, 0.25, 0.5, 0.5, 1.0, 0.25, 0.25, 0.25, 0.25, 0.5, 0.5, 0.25, 0.25, 1.0, 1.0, 0.5, 0.25, 0.25, 0.5, 0.5, 1.0, 0.5, 0.25, 0.25, 0.5, 0.5, 0.25, 0.25, 0.5, 0.5, 0.5, 0.5, 0.5, 1.0, 0.5]
[0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0]


In [8]:
phrases_length = dict()
counter = 0
for i in range(len(phrases_data)):
    counter+=1
    if phrases_data[i] == 1:
        if counter in phrases_length.keys():
            phrases_length[counter] += 1
        else:
            phrases_length[counter] = 1
        counter = 0

phrases_length = sorted(phrases_length.items(), key=lambda x: x[1], reverse=True)
phrases_length = [ (length, count) for length, count in phrases_length if count > 0.05 * number_of_pieces]
phrases_length

[(14, 5605),
 (16, 5309),
 (12, 5131),
 (13, 4215),
 (8, 4181),
 (15, 3808),
 (10, 3332),
 (18, 3300),
 (17, 3220),
 (11, 2924),
 (9, 2878),
 (19, 2362),
 (7, 2307),
 (20, 2035),
 (21, 1337),
 (6, 1211),
 (22, 1063)]

Remove pieces with irregular phrases

In [9]:
seq_len = phrases_length[0][0]
lower_bound = min([ length for length, count in phrases_length ])
upper_bound = max([ length for length, count in phrases_length ])

In [10]:
# The most common phrases length ==> the lenght of a sequence
seq_len

14

In [11]:
# The shortest phrase lenght we consider
lower_bound

6

In [12]:
# The longest phrase lenght we consider
upper_bound

22

In [13]:
pieces = []

for piece in data:
    # Skip pieces with irregular phrases
    current_phrase_length = 0
    skip = False

    for i, digit in enumerate(piece[3]):
        current_phrase_length += 1
        if digit == 1:
            if current_phrase_length < lower_bound or current_phrase_length > upper_bound:
                skip = True
                break
            current_phrase_length = 0
    if skip:
        continue
    else:
        pieces.append(piece)

In [14]:
number_of_pieces = len(pieces)
number_of_pieces

16809

### Split the data into train and test data

In [15]:
# Train test split
pieces_train, pieces_test = train_test_split(pieces, test_size=0.2, random_state=42)

In [16]:
len(pieces_train), len(pieces_test)

(13447, 3362)

In [17]:
# save the train and test data to pickle files
with open('pieces_train.pkl', 'wb') as handle:
    pickle.dump(pieces_train, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('pieces_test.pkl', 'wb') as handle:
    pickle.dump(pieces_test, handle, protocol=pickle.HIGHEST_PROTOCOL)

### Extract train data

In [18]:
notes_input = []
durations_input = []
bars_input = []
phrases_input = []

for piece in pieces_train:
    notes = piece[0]
    notes_input.append(129) # START = 129
    notes_input.extend(notes)

    durations = piece[1]
    durations_input.append(0)
    durations_input.extend(durations)

    bars = piece[2]
    bars_input.append(0)
    bars_input.extend(bars)
    
    phrases = piece[3]
    phrases_input.append(0)
    phrases_input.extend(phrases)

In [19]:
print("Length of the notes_input: {}".format(len(notes_input)))
print("Twenty first elements: {}".format(notes_input[:20]))
print("Length of the durations_input: {}".format(len(durations_input)))
print("Twenty first elements: {}".format(durations_input[:20]))
print("Length of the phrases_input: {}".format(len(phrases_input)))
print("Twenty first elements: {}".format(phrases_input[:20]))

Length of the notes_input: 553633
Twenty first elements: [129, 60, 67, 60, 68, 67, 62, 67, 62, 65, 63, 67, 63, 60, 68, 67, 67, 59, 60, 67]
Length of the durations_input: 553633
Twenty first elements: [0, 0.5, 0.75, 0.25, 0.5, 1.0, 0.5, 0.75, 0.25, 0.5, 1.0, 0.5, 0.5, 0.5, 0.5, 1.0, 1.0, 0.5, 1.5, 0.5]
Length of the phrases_input: 553633
Twenty first elements: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0]


### Prepare data for embedding

Map durations and notes to integers

In [20]:
durations_to_int = {}
int_to_duration = {}
notes_to_int = {}
int_to_notes = {}
phrases_to_int = {}
int_to_phrases = {}
bars_to_int = {}
int_to_bars = {}

for index, duration in enumerate(sorted(set(durations_data).union({0}))):
    durations_to_int[duration] = index
    int_to_duration[index] = duration

for index, note in enumerate(sorted(set(notes_data).union({129}))):
    notes_to_int[note] = index
    int_to_notes[index] = note

for index, phrase in enumerate(sorted(set(phrases_data))):
    phrases_to_int[phrase] = index
    int_to_phrases[index] = phrase

for index, bar in enumerate(sorted(set(bars_data))):
    bars_to_int[bar] = index
    int_to_bars[index] = bar

durations_network_input = [ durations_to_int[duration] for duration in durations_input ]
notes_network_input = [ notes_to_int[note] for note in notes_input ]
bars_network_input = [ bars_to_int[bar] for bar in bars_input ]
phrases_network_input = [ phrases_to_int[phrase] for phrase in phrases_input ]

In [21]:
# Before
print(sorted(set(durations_input)))

[0, 0.00390625, 0.0625, 0.125, 0.25, 0.375, 0.5, 0.75, 1.0, 1.5, 2.0, 3.0, 4.0]


In [22]:
# After
print(sorted(set(durations_network_input)))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


In [23]:
# Before
print(sorted(set(notes_input)))

[43, 46, 47, 48, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 86, 88, 128, 129]


In [24]:
# After
print(sorted(set(notes_network_input)))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 41, 44, 45]


In [25]:
print("Length of the notes_input: {}".format(len(notes_network_input)))
print("Twenty first elements: {}".format(notes_network_input[:20]))
print("Length of the durations_input: {}".format(len(durations_network_input)))
print("Twenty first elements: {}".format(durations_network_input[:20]))
print("Length of the phrases_input: {}".format(len(phrases_network_input)))
print("Twenty first elements: {}".format(phrases_network_input[:20]))

Length of the notes_input: 553633
Twenty first elements: [45, 14, 21, 14, 22, 21, 16, 21, 16, 19, 17, 21, 17, 14, 22, 21, 21, 13, 14, 21]
Length of the durations_input: 553633
Twenty first elements: [0, 6, 7, 4, 6, 8, 6, 7, 4, 6, 8, 6, 6, 6, 6, 8, 8, 6, 9, 6]
Length of the phrases_input: 553633
Twenty first elements: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0]


In [26]:
n_notes = len(notes_to_int)
n_durations = len(durations_to_int)
n_phrases = len(phrases_to_int)
n_bars = len(bars_to_int)

In [27]:
print("Number of distinct notes: {}".format(n_notes))
print("Number of distinct durations: {}".format(n_durations))

Number of distinct notes: 46
Number of distinct durations: 13


In [28]:
with open('dictionary.pkl', 'wb') as file:
    dictionary = { "durations_to_int" : durations_to_int, "int_to_duration" : int_to_duration, "notes_to_int" : notes_to_int, "int_to_notes" : int_to_notes, "phrases_to_int" : phrases_to_int, "int_to_phrases" : int_to_phrases, "bars_to_int" : bars_to_int, "int_to_bars" : int_to_bars }
    # A new file will be created
    pickle.dump(dictionary, file)

### Prepare network input and output

In [29]:
def prepare_sequences(notes, durations, bars, phrases, seq_len = 32):

    notes_input = []
    durations_input = []
    bars_input = []
    phrases_output = []

    for i in range(len(notes) - seq_len):
        notes_input.append(notes[i:i + seq_len])
        durations_input.append(durations[i:i + seq_len])
        bars_input.append(bars[i:i + seq_len])

        phrases_output.append(phrases[i + seq_len])

    n_patterns = len(notes_input)

    notes_input = np.reshape(notes_input, (n_patterns, seq_len))
    durations_input = np.reshape(durations_input, (n_patterns, seq_len))
    bars_input = np.reshape(bars_input, (n_patterns, seq_len))
    network_input = [notes_input, durations_input, bars_input]

    phrases_output = np_utils.to_categorical(phrases_output, num_classes=2)
    network_output = [phrases_output]

    return (network_input, network_output)

In [30]:
network_input, network_output = prepare_sequences(notes_network_input, durations_network_input, bars_network_input, phrases_network_input, seq_len)

In [31]:
print('note input')
print(network_input[0][0])
print('duration input')
print(network_input[1][0])
print('bars input')
print(network_input[2][0])
print('phrases_output')
print(network_output[0][0])

note input
[45 14 21 14 22 21 16 21 16 19 17 21 17 14]
duration input
[0 6 7 4 6 8 6 7 4 6 8 6 6 6]
bars input
[0 0 0 1 0 1 0 0 1 0 1 0 0 1]
phrases_output
[1. 0.]


### Create neural network

In [32]:
def create_network(n_notes, n_durations, n_bars, n_phrases, embed_size = 100, rnn_units = 256, use_attention = False):

    notes_in = Input(shape = (None,))
    durations_in = Input(shape = (None,))
    bars_in = Input(shape = (None,))

    x1 = Embedding(n_notes, embed_size)(notes_in)
    x2 = Embedding(n_durations, embed_size)(durations_in)
    x3 = Embedding(n_bars, embed_size)(bars_in)

    x = Concatenate()([x1,x2,x3])

    x = LSTM(rnn_units, return_sequences=True)(x)

    if use_attention:

        x = LSTM(rnn_units, return_sequences=True)(x)

        e = Dense(1, activation='tanh')(x)
        e = Reshape([-1])(e)
        alpha = Activation('softmax')(e)

        alpha_repeated = Permute([2, 1])(RepeatVector(rnn_units)(alpha))

        c = Multiply()([x, alpha_repeated])
        c = Lambda(lambda xin: K.sum(xin, axis=1), output_shape=(rnn_units,))(c)
    
    else:
        c = LSTM(rnn_units)(x)
                                    
    phrases_out = Dense(n_phrases, activation = 'softmax', name = 'phrase')(c)
   
    model = Model([notes_in, durations_in, bars_in], [phrases_out])
    

    if use_attention:
        att_model = Model([notes_in, durations_in, bars_in], alpha)
    else:
        att_model = None


    opti = RMSprop(lr = 0.001)
    model.compile(loss=['binary_crossentropy', 'binary_crossentropy'], optimizer=opti)

    return model, att_model

In [33]:
embed_size = 46
rnn_units = 512
use_attention = True
model, att_model = create_network(n_notes, n_durations, n_bars, n_phrases, embed_size, rnn_units, use_attention)
model.summary()

2023-05-21 22:25:01.650977: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 46)     2116        ['input_1[0][0]']                
                                                                                              

/opt/conda/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/rmsprop.py:143: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


### Train the model

In [ ]:
weights_folder = "weights"

checkpoint1 = ModelCheckpoint(
    os.path.join(weights_folder, "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.h5"),
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)

checkpoint2 = ModelCheckpoint(
    os.path.join(weights_folder, "weights.h5"),
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)

early_stopping = EarlyStopping(
    monitor='loss'
    , restore_best_weights=True
    , patience = 10
)


callbacks_list = [
    checkpoint1
    , checkpoint2
    , early_stopping
 ]

model.save_weights(os.path.join(weights_folder, "weights.h5"))
model.fit(network_input, network_output
          , epochs=30, batch_size=200
          , validation_split = 0.2
          , callbacks=callbacks_list
          , shuffle=True
         )

Epoch 1/30
2215/2215 [==============================] - 625s 280ms/step - loss: 0.2236 - val_loss: 0.2031
Epoch 2/30
2215/2215 [==============================] - 680s 307ms/step - loss: 0.1810 - val_loss: 0.1722
Epoch 3/30
2215/2215 [==============================] - 745s 336ms/step - loss: 0.1675 - val_loss: 0.1698
Epoch 4/30
2215/2215 [==============================] - 714s 322ms/step - loss: 0.1574 - val_loss: 0.1637
Epoch 5/30
2215/2215 [==============================] - 719s 325ms/step - loss: 0.1448 - val_loss: 0.1627
Epoch 6/30
2215/2215 [==============================] - 735s 332ms/step - loss: 0.1286 - val_loss: 0.1684
Epoch 7/30
2215/2215 [==============================] - 718s 324ms/step - loss: 0.1091 - val_loss: 0.1758
Epoch 8/30
2215/2215 [==============================] - 729s 329ms/step - loss: 0.0894 - val_loss: 0.2033
Epoch 9/30
2215/2215 [==============================] - 747s 337ms/step - loss: 0.0709 - val_loss: 0.2217
Epoch 10/30
2215/2215 [=======================